In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from time import time
from datetime import datetime
from PIL import Image
import argparse
from collections import defaultdict

from keras import layers, models, optimizers
from keras import backend as K

K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks

from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.losses import categorical_crossentropy

from keras_tqdm import TQDMCallback

from utils import plot_log,save_results_to_csv

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask

sys.path.append('./PatchyTools/')
from GraphConverter import GraphConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from ConvNetPatchy import AccuracyHistory
from GraphClassifier import GraphClassifier

DIR_PATH = os.environ['GAMMA_DATA_ROOT']
GRAPH_RELABEL_NAME = '_relabeled'
RESULTS_PATH = os.path.join(DIR_PATH, 'Results/CapsuleSans/CNN_Caps_comparison.csv')



### Load data

In [ ]:
# Getting the data:
dataset_name = 'PROTEINS'
receptive_field = 10
relabelling = False
epochs = 100
n_folds = 2

In [ ]:
# Converting Graphs into Matrices:
PatchyConverter = GraphConverter(dataset_name, receptive_field, GRAPH_RELABEL_NAME)
if relabelling:
    PatchyConverter.relabel_graphs()

graph_tensor = PatchyConverter.graphs_to_Patchy_tensor()
avg_nodes_per_graph = PatchyConverter.avg_nodes_per_graph

# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)


In [ ]:
# Capsule Architecture Parameters:
capsule_params = CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params, conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2, 2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params, caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params, digit_layer_name)

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256  # 250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params, decoder_layer)

# Training Hyperparameters:

args_train = CapsuleTrainingParameters()
args_train.verbose = 0
if not os.path.exists(args_train.save_dir):
    os.makedirs(args_train.save_dir)

In [ ]:
# Generate list of parameter sets::
list_parameter_sets = []
#list_parameter_sets.append(args_train)

n_epoch_list = [epochs]  # , 150, 200]
lr_list = [0.001]  # , 0.005]#[0.0005, 0.001]#, 0.005]
lr_decay_list = [0.9]  # , 0.75]#[0.25,0.4]#, 0.75, 1.5]
lam_recon_list = [0.392]  # ,0.7] # [0.25,0.4,0.55]

for n_epoch in n_epoch_list:
    for lr in lr_list:
        for lr_decay in lr_decay_list:
            for lam_recon in lam_recon_list:
                training_params = CapsuleTrainingParameters(epochs=n_epoch,
                                                            lr=lr,
                                                            lr_decay=lr_decay,
                                                            lam_recon=lam_recon)
                list_parameter_sets.append(training_params)

#list_parameter_sets[-1].plot_log = True

# Default parameters:
print('Training in {} parameter sets'.format(len(list_parameter_sets)))


In [ ]:
fold_set = []

for j in range(n_folds):
    fold_set.append(train_test_split(graph_tensor,
                                     graph_labels,
                                     test_size=0.10,
                                     random_state=j))
results_df = []
val_acc = []
for i, parameter_set in enumerate(list_parameter_sets):
    for j in range(n_folds):

        x_train, x_test, y_train, y_test = fold_set[j]
        data = ((x_train, y_train), (x_test, y_test))
        input_shape = x_train.shape[1:]
        parameter_set.add_fold(i)

        patchy_classifier = GraphClassifier(input_shape)
        patchy_classifier.build_the_graph(capsule_params)
        patchy_classifier.train(data, parameter_set)


        val_acc.append(patchy_classifier.results.val_capsnet_acc)

        #if i == 0:
        results_df.append(pd.DataFrame(patchy_classifier.results))
        #else:
        print('Set of parameters {} trained '.format(i + 1))

In [ ]:
score = patchy_classifier.train_model.evaluate([[patchy_classifier.x_test, patchy_classifier.y_test],patchy_classifier.y_test, patchy_classifier.x_test], verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plt.plot(range(patchy_cnn.epochs), patchy_cnn.history.acc)
plt.legend('train data',loc='lower center')
plt.plot(range(patchy_cnn.epochs), patchy_cnn.history.val_acc)
plt.legend('test data')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()